<a href="https://colab.research.google.com/github/ArpitaChatterjee/Recommendation-System/blob/main/Recommendation_System.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**To build this Recommemdation system , I've used Collaborative Filtering**

In [1]:
#necessary inputs
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sklearn.metrics as metrics
from sklearn.neighbors import NearestNeighbors
from scipy.spatial.distance import correlation , cosine
import ipywidgets as widgets
from IPython.display import display, clear_output
from sklearn.metrics import pairwise_distances
from sklearn.metrics import mean_squared_error
from math import sqrt
from contextlib import contextmanager

Given a user-item ratings matrix M below, where 6 users (rows) have rated 6 items (columns). Ratings can take up integer values from 1–10 and 0 indicates absence of rating. (Here I'm using zero-based indexing for rows and columns as used in Python. However, for user input, user_id will take up numbers from 1–6 and item_id from 1–6). Suppose, I’ve to find if user 3 will like item 4 or not. Hence user 3 becomes the target user or active user and item 4 is the target item.

In [2]:
#matrix to store user-item ratings(from 1-10) based from which recommemdation system can be developed
m = np.asarray([[4,5,6,7,7,9],
                [7,0,7,9,9,8],
                [7,5,5,0,8,4],
               [5,6,8,5,9,8],
               [5,8,8,8,10,9],
               [7,7,0,4,7,8]])

#converting into dataframe
m=pd.DataFrame(m)


In [3]:
#declaring global varible which can be changed later
global k, metric 
k=4
metric= 'cosine' #for pearson correlation similarities-> 'correlation' 

In [4]:
m

,0,1,2,3,4,5
0,4,5,6,7,7,9
1,7,0,7,9,9,8
2,7,5,5,0,8,4
3,5,6,8,5,9,8
4,5,8,8,8,10,9
5,7,7,0,4,7,8


**User-Based Colaborative Filtering**

In [5]:
#cosine similarities for rating matrix 'm'
cosine_s= 1-pairwise_distances(m, metric="cosine")
#pairwise_distance returns distance/ difference btw ratings; 
#thus similarity is 1-distance

In [6]:
#cosine Similarity matrix
pd.DataFrame(cosine_s)

,0,1,2,3,4,5
0,1.000000,0.930556,0.817507,0.975223,0.986845,0.879433
1,0.930556,1.000000,0.780654,0.908914,0.905043,0.781719
2,0.817507,0.780654,1.000000,0.909513,0.865454,0.853275
3,0.975223,0.908914,0.909513,1.000000,0.989344,0.865614
4,0.986845,0.905043,0.865454,0.989344,1.000000,0.881640
5,0.879433,0.781719,0.853275,0.865614,0.881640,1.000000


In [7]:
#Pearson Similarity for rating matrix 'm'
pearson_s =1- pairwise_distances(m, metric='correlation')

In [8]:
#Pearson Correlation similarity matrix 
pd.DataFrame(pearson_s)

,0,1,2,3,4,5
0,1.000000,0.494663,-0.355166,0.552562,0.750776,0.113579
1,0.494663,1.000000,-0.113030,0.262896,0.176483,-0.156632
2,-0.355166,-0.113030,1.000000,0.451378,-0.042888,0.297373
3,0.552562,0.262896,0.451378,1.000000,0.763325,-0.057739
4,0.750776,0.176483,-0.042888,0.763325,1.000000,0.039621
5,0.113579,-0.156632,0.297373,-0.057739,0.039621,1.000000


In [9]:
#function to find k similar users given the userid and rating matrix m
#findksimilarusers function uses this method to return similarities and indices of k-nearest neighbors for active user.
def findksimilarusers(userid, ratings, metric= metric, k=k):
  similar=[]
  index=[]
  model= NearestNeighbors(metric= metric, algorithm='brute')
  model.fit(ratings)

  distances, index = model.kneighbors(ratings.iloc[userid-1, :].values.reshape(1,-1), n_neighbors= k+1)
  similar= 1-distances.flatten()
  print ("{} most similar users for User {}:".format(k,userid))
  for i in range(0, len(index.flatten())):
    if index.flatten()[i]+1 == userid:
      continue
    else:
      print('{}:User{}, with similarity of {}'.format(i, index.flatten()[i]+1, similar.flatten()[i]))
  return similar, index 

In [10]:
similar, index= findksimilarusers(1, m, metric='cosine') 

4 most similar users for User 1:
1:User5, with similarity of 0.9868452046367622
2:User4, with similarity of 0.9752227038137999
3:User2, with similarity of 0.9305555555555556
4:User6, with similarity of 0.879433387869963


In [11]:
similar , index = findksimilarusers(1, m, metric='correlation')

4 most similar users for User 1:
1:User5, with similarity of 0.7507759960689286
2:User4, with similarity of 0.5525620921281691
3:User2, with similarity of 0.49466312545461666
4:User6, with similarity of 0.11357879791107994


In [12]:
#a funtion to predict rating for specified user-item combination based on user-based approach
def pred_userbased(userid, itemid, ratings, metric= metric, k=k):
  prediction= 0
  similar, index= findksimilarusers(userid, ratings, metric, k)#similar uses based on cosine similarity
  mean_rating = ratings.loc[userid-1,:].mean()#to adjust for zero based indexing
  sum_wt= np.sum(similar)-1
  product=1
  wtd_sum=0

  for i in range(0, len(index.flatten())):
    if index.flatten()[i]+1== userid:
      continue
    else:
      rating_diff= ratings.iloc[index.flatten()[i], itemid-1]- np.mean(ratings.iloc[index.flatten()[i],:])
      product= rating_diff*(similar[i])
      wtd_sum= wtd_sum+product
    
  prediction= int(round(mean_rating+ (wtd_sum/sum_wt)))
  print('\nPredicted rating for user{}-> item{}:{}'.format(userid, itemid, prediction))

  return prediction


In [13]:
prediction = pred_userbased(3, 4, m)

4 most similar users for User 3:
1:User4, with similarity of 0.909512689340191
2:User5, with similarity of 0.8654538781497916
3:User6, with similarity of 0.853274963343837
4:User1, with similarity of 0.8175071326974299

Predicted rating for user3-> item4:4


**Item-Based Recommendation System**

In this process, similarities between pair of items are computed using cosine similarity metric

In [14]:
#funtn to find k similar items given the itemid n ratings matrix m
def findksimilaritems(itemid, ratings, metric= metric, k=k):
  similar=[]
  index=[]
  ratings=ratings.T
  model= NearestNeighbors(metric= metric, algorithm='brute')
  model.fit(ratings)

  distances, index = model.kneighbors(ratings.iloc[itemid-1, :].values.reshape(1,-1), n_neighbors= k+1)
  similar= 1-distances.flatten()
  print ("{} most similar items for item {}:".format(k,itemid))
  for i in range(0, len(index.flatten())):
    if index.flatten()[i]+1 == itemid:
      continue
    else:
      print('{}:Item{}, with similarity of {}'.format(i, index.flatten()[i]+1, similar.flatten()[i]))
  return similar, index 

In [15]:
similar, index = findksimilaritems(3, m)

4 most similar items for item 3:
1:Item5, with similarity of 0.9349428022311271
2:Item6, with similarity of 0.8963802849585222
3:Item4, with similarity of 0.8837387055954756
4:Item1, with similarity of 0.8349867213266859


In [16]:
def pred_itembased(user_id, item_id, ratings, metric = metric, k=k):
    prediction= wtd_sum =0
    similarities, indices=findksimilaritems(item_id, ratings) #similar users based on correlation coefficients
    sum_wt = np.sum(similarities)-1
    product=1
    
    for i in range(0, len(indices.flatten())):
        if indices.flatten()[i]+1 == item_id:
            continue;
        else:
            product = ratings.iloc[user_id-1,indices.flatten()[i]] * (similarities[i])
            wtd_sum = wtd_sum + product                              
    prediction = int(round(wtd_sum/sum_wt))
    print ('\nPredicted rating for user {0} -> item {1}: {2}'.format(user_id,item_id,prediction))   

    return prediction

In [17]:
prediction= pred_itembased(1,3,m)

4 most similar items for item 3:
1:Item5, with similarity of 0.9349428022311271
2:Item6, with similarity of 0.8963802849585222
3:Item4, with similarity of 0.8837387055954756
4:Item1, with similarity of 0.8349867213266859

Predicted rating for user 1 -> item 3: 7


In [18]:
#funtion to compute adjusted cosine similarity matrix for items
def computeAdjCosSim(M):
    sim_matrix = np.zeros((M.shape[1], M.shape[1]))
    M_u = M.mean(axis=1) #means
          
    for i in range(M.shape[1]):
        for j in range(M.shape[1]):
            if i == j:
                
                sim_matrix[i][j] = 1
            else:                
                if i<j:
                    
                    sum_num = sum_den1 = sum_den2 = 0
                    for k,row in M.loc[:,[i,j]].iterrows(): 

                        if ((M.loc[k,i] != 0) & (M.loc[k,j] != 0)):
                            num = (M[i][k]-M_u[k])*(M[j][k]-M_u[k])
                            den1= (M[i][k]-M_u[k])**2
                            den2= (M[j][k]-M_u[k])**2
                            
                            sum_num = sum_num + num
                            sum_den1 = sum_den1 + den1
                            sum_den2 = sum_den2 + den2
                        
                        else:
                            continue                          
                                       
                    den=(sum_den1**0.5)*(sum_den2**0.5)
                    if den!=0:
                        sim_matrix[i][j] = sum_num/den
                    else:
                        sim_matrix[i][j] = 0


                else:
                    sim_matrix[i][j] = sim_matrix[j][i]           
            
    return pd.DataFrame(sim_matrix)

In [19]:
adjcos_sim= computeAdjCosSim(m)
adjcos_sim

,0,1,2,3,4,5
0,1.000000,0.668657,-0.147273,0.021888,-0.063420,-0.421862
1,0.668657,1.000000,-0.258199,-0.301283,0.008262,-0.103806
2,-0.147273,-0.258199,1.000000,-0.413728,0.573472,0.177300
3,0.021888,-0.301283,-0.413728,1.000000,-0.023958,-0.070491
4,-0.063420,0.008262,0.573472,-0.023958,1.000000,0.476217
5,-0.421862,-0.103806,0.177300,-0.070491,0.476217,1.000000


In [20]:
#function to find k similar items given the item_id and ratings matrix M

def findksimilaritems_adjcos(item_id, ratings, k=k):
    
    sim_matrix = computeAdjCosSim(ratings)
    similar = sim_matrix[item_id-1].sort_values(ascending=False)[:k+1].values
    index = sim_matrix[item_id-1].sort_values(ascending=False)[:k+1].index
    
    print ('{} most similar items for item {}:\n'.format(k,item_id))
    for i in range(0, len(index)):
            if index[i]+1 == item_id:
                continue;

            else:
                print ('{}: Item {} :, with similarity of {}'.format(i,index[i]+1, similar[i]))
        
    return similar ,index

In [21]:
similar, index =findksimilaritems_adjcos(3, m)

4 most similar items for item 3:

1: Item 5 :, with similarity of 0.5734723650156127
2: Item 6 :, with similarity of 0.17730043730392672
3: Item 1 :, with similarity of -0.14727321888159384
4: Item 2 :, with similarity of -0.2581988897471612


In [22]:
#As the adjusted cosine similarities range from -1,+1, sometimes the predicted rating can be negative or greater than max value
#This function predicts the rating for specified user-item combination for adjusted cosine item-based approach; where  Rating is set to min if prediction is negative, Rating is set to max if prediction is above max
def predict_itembased_adjcos(user_id, item_id, ratings):
    prediction=0

    similarities, indices=findksimilaritems_adjcos(item_id, ratings) #similar users based on correlation coefficients
    sum_wt = np.sum(similarities)-1

    product=1
    wtd_sum = 0 
    for i in range(0, len(indices)):
        if indices[i]+1 == item_id:
            continue;
        else:
            product = ratings.iloc[user_id-1,indices[i]] * (similarities[i])
            wtd_sum = wtd_sum + product                              
    prediction = int(round(wtd_sum/sum_wt))
    if prediction < 0:
        prediction = 1
    elif prediction >10:
        prediction = 10
    print ('\nPredicted rating for user {0} -> item {1}: {2}'.format(user_id,item_id,prediction))
        
    return prediction

In [23]:
prediction = predict_itembased_adjcos(3, 4,m)

4 most similar items for item 4:

1: Item 1 :, with similarity of 0.02188833692468515
2: Item 5 :, with similarity of -0.023957680199099374
3: Item 6 :, with similarity of -0.0704907376850242
4: Item 2 :, with similarity of -0.30128345711746257

Predicted rating for user 3 -> item 4: 5


In [24]:
adjcos_sim

,0,1,2,3,4,5
0,1.000000,0.668657,-0.147273,0.021888,-0.063420,-0.421862
1,0.668657,1.000000,-0.258199,-0.301283,0.008262,-0.103806
2,-0.147273,-0.258199,1.000000,-0.413728,0.573472,0.177300
3,0.021888,-0.301283,-0.413728,1.000000,-0.023958,-0.070491
4,-0.063420,0.008262,0.573472,-0.023958,1.000000,0.476217
5,-0.421862,-0.103806,0.177300,-0.070491,0.476217,1.000000


In [25]:
#function utilizes above function to recommend items for selected approach. Recommendations are made if the predicted
#rating for an item is greater than or equal to 6, and the items has not been rated already
def recommendItem(user_id, item_id, ratings):
    
    if user_id<1 or user_id>6 or type(user_id) is not int:
        print ('Userid does not exist. Enter numbers from 1-6')
    else:    
        ids = ['User-based CF (cosine)','User-based CF (correlation)','Item-based CF (cosine)',
               'Item-based CF (adjusted cosine)']

        approach = widgets.Dropdown(options=ids, value=ids[0],
                               description='Select Approach', width='500px')
        
        def on_change(change):
            prediction = 0
            clear_output(wait=True)
            if change['type'] == 'change' and change['name'] == 'value':            
                if (approach.value == 'User-based CF (cosine)'):
                    metric = 'cosine'
                    prediction = pred_userbased(user_id, item_id, ratings, metric)
                elif (approach.value == 'User-based CF (correlation)')  :                       
                    metric = 'correlation'               
                    prediction = pred_userbased(user_id, item_id, ratings, metric)
                elif (approach.value == 'Item-based CF (cosine)'):
                    prediction = pred_itembased(user_id, item_id, ratings)
                else:
                    prediction = predict_itembased_adjcos(user_id,item_id,ratings)

                if ratings[item_id-1][user_id-1] != 0: 
                    print ('Item already rated')
                else:
                    if prediction>=6:
                        print ('\nItem recommended')
                    else:
                        print ('Item not recommended')

        approach.observe(on_change)
        display(approach)

In [26]:
#check for incorrect entries
recommendItem(-1,3,m)

Userid does not exist. Enter numbers from 1-6


In [27]:
recommendItem(3,4,m) #userbased correation

4 most similar users for User 3:
1:User4, with similarity of 0.45137800509830084
2:User6, with similarity of 0.29737330482459035
3:User5, with similarity of -0.042887787939969835
4:User2, with similarity of -0.11302975767633616

Predicted rating for user3-> item4:2
Item not recommended


In [28]:
recommendItem(3,4,m)#itebased cosine

4 most similar items for item 4:
1:Item6, with similarity of 0.9461705147055777
2:Item5, with similarity of 0.8965398044904246
3:Item3, with similarity of 0.8837387055954756
4:Item1, with similarity of 0.822420808597973

Predicted rating for user 3 -> item 4: 6

Item recommended


In [29]:
recommendItem(3,4,m)#itembased ajdcos

4 most similar items for item 4:

1: Item 1 :, with similarity of 0.02188833692468515
2: Item 5 :, with similarity of -0.023957680199099374
3: Item 6 :, with similarity of -0.0704907376850242
4: Item 2 :, with similarity of -0.30128345711746257

Predicted rating for user 3 -> item 4: 5
Item not recommended


In [30]:
recommendItem(2,1,m)#if item is already rated its not recommended

4 most similar users for User 2:
1:User1, with similarity of 0.49466312545461666
2:User4, with similarity of 0.262895775278958
3:User5, with similarity of 0.17648280200062938
4:User3, with similarity of -0.11302975767633616

Predicted rating for user2-> item1:4
Item already rated


In [31]:
#This is a quick way to temporarily suppress stdout in particular code section
import sys, os
from contextlib import contextmanager

@contextmanager
def suppress_stdout():
    with open(os.devnull, "w") as devnull:
        old_stdout = sys.stdout
        sys.stdout = devnull
        try:  
            yield
        finally:
            sys.stdout = old_stdout


In [32]:
#This is final function to evaluate the performance of selected recommendation approach and the metric used here is RMSE
#suppress_stdout function is used to suppress the print outputs of all the functions inside this function. It will only print 
#RMSE values
def evaluateRS(ratings):
    ids = ['User-based CF (cosine)','User-based CF (correlation)','Item-based CF (cosine)','Item-based CF (adjusted cosine)']
    approach = widgets.Dropdown(options=ids, value=ids[0],description='Select Approach', width='500px')
    n_users = ratings.shape[0]
    n_items = ratings.shape[1]
    prediction = np.zeros((n_users, n_items))
    prediction= pd.DataFrame(prediction)
    def on_change(change):
        clear_output(wait=True)
        with suppress_stdout():
            if change['type'] == 'change' and change['name'] == 'value':            
                if (approach.value == 'User-based CF (cosine)'):
                    metric = 'cosine'
                    for i in range(n_users):
                        for j in range(n_items):
                            prediction[i][j] = pred_userbased(i+1, j+1, ratings, metric)
                elif (approach.value == 'User-based CF (correlation)')  :                       
                    metric = 'correlation'               
                    for i in range(n_users):
                        for j in range(n_items):
                            prediction[i][j] = pred_userbased(i+1, j+1, ratings, metric)
                elif (approach.value == 'Item-based CF (cosine)'):
                    for i in range(n_users):
                        for j in range(n_items):
                            prediction[i][j] = pred_userbased(i+1, j+1, ratings)
                else:
                    for i in range(n_users):
                        for j in range(n_items):
                            prediction[i][j] = pred_userbased(i+1, j+1, ratings)
              
        MSE = mean_squared_error(prediction, ratings)
        RMSE = round(sqrt(MSE),3)
        print ("RMSE using {0} approach is: {1}".format(approach.value,RMSE))
              
    approach.observe(on_change)
    display(approach)

In [33]:
evaluateRS(m)

RMSE using User-based CF (correlation) approach is: 2.981


In [34]:
evaluateRS(m)

RMSE using Item-based CF (cosine) approach is: 2.92
